In [10]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

def rossler(t, state, a, b, c):
    x, y, z = state
    dxdt = -y - z
    dydt = x + a * y
    dzdt = b + z * (x - c)
    return [dxdt, dydt, dzdt]


def simulate_rossler(a, b, c, initial_state, t_span=(0, 10), dt=0.01):
    t_eval = np.arange(t_span[0], t_span[1], dt)
    sol = solve_ivp(
        rossler, t_span, initial_state, args=(a, b, c),
        t_eval=t_eval, rtol=1e-8, atol=1e-10
    )
    return sol.t, sol.y

def plot_system(system='Rossler', param=28.0):
    
    a = 0.1
    b = 0.1
    c = param
    initial_state1 = [-4.0, -4.0, 0.0]
    #t1, states1 = simulate_rossler(a, b, c, initial_state1, t_span=(0, 200))
    t1, states1 = simulate_rossler(a, b, c, initial_state1, t_span=(0, 200), dt=0.005)
    states1 = states1[:, ::20]  # Downsample to match data generation
    title = f"Rössler System (a={a}, b={b}, c={c:.2f})"
    label1 = 'Initial: [1.0, 1.0, 1.0]'
    label2 = 'Initial: [-1.0, -1.0, -1.0]'

    fig = plt.figure(figsize=(10, 7))
    ax = fig.add_subplot(111, projection='3d')
    ax.plot(states1[0], states1[1], states1[2], color='mediumpurple', label=label1)
    ax.set_title(title)
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.set_zlabel("Z")
    ax.set_xlim([-10, 15])
    ax.set_ylim([-10, 0])
    ax.set_zlim([0, 15])
    ax.legend()
    plt.show()

interact(
    plot_system,
    system={ 'Rössler': 'rossler'},
    param=FloatSlider(min=0, max=12, step=0.5, value=4, description='r')
)

interactive(children=(Dropdown(description='system', options={'Rössler': 'rossler'}, value='rossler'), FloatSl…

<function __main__.plot_system(system='Rossler', param=28.0)>

In [35]:
import os
import torch

def simulate_rossler(a, b, c, initial_state, t_span=(0, 200), dt=0.01):
    t_eval = np.arange(t_span[0], t_span[1] + dt, dt)
    sol = solve_ivp(
        rossler, t_span, initial_state, args=(a, b, c),
        t_eval=t_eval, rtol=1e-8, atol=1e-10
    )
    return sol.t, sol.y

def save_trajectory_plot_with_samples(states, t, c, initial_state, sample_t, save_dir, idx_traj):
    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111, projection='3d')
    # Plot full trajectory
    ax.plot(states[0], states[1], states[2], label='Full trajectory')
    # Plot sampled points
    sample_indices = [np.argmin(np.abs(t - st)) for st in sample_t]
    ax.scatter(states[0][sample_indices], states[1][sample_indices], states[2][sample_indices],
               color='red', s=50, label='Sampled points')
    ax.set_title(f"Rössler (c={c}, init={initial_state})")
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.set_zlabel("Z")
    ax.set_xlim([-10, 15])
    ax.set_ylim([-15, 0])
    ax.set_zlim([0, 20])
    ax.legend()
    plt.tight_layout()
    fname = f"rossler_traj{idx_traj}_c{c}_init{initial_state}.png"
    plt.savefig(os.path.join(save_dir, fname))
    plt.close(fig)

def generate_rossler_data_at_timepoints(c_list, initial_state, sample_t, t_span=(0, 200), dt=0.01, a=0.1, b=0.1):
    save_dir = "rossler_trajectories"
    os.makedirs(save_dir, exist_ok=True)
    num_c = len(c_list)
    num_samples = len(sample_t)
    data = torch.zeros((num_samples, num_c, 3), dtype=torch.float32)

    for idx, c in enumerate(c_list):
        t, states = simulate_rossler(a, b, c, initial_state, t_span, dt)
        sample_indices = [np.argmin(np.abs(t - st)) for st in sample_t]
        sampled_states = states[:, sample_indices].T  # shape: (num_samples, 3)
        data[:, idx, :] = torch.tensor(sampled_states, dtype=torch.float32)
        save_trajectory_plot_with_samples(states, t, c, initial_state, sample_t, save_dir, idx)
    return data

# Example usage:
initial_state = [-4.0, -4.0, 0.0]  # Initial state for the Rössler system
c_list = [3,3.5,4,4.5,5,5.5,6,6.5,7,8,8.5]  # Your desired c values
sample_t = np.arange(900, 1000, 1)  # Sample time points from 200 to 300 with step 0.1
data = generate_rossler_data_at_timepoints(c_list, initial_state, sample_t, t_span=(0, 1000), dt=0.01)
print(data.shape)

#Save the data and the time points in a .pth file
torch.save({'data': data, 'c_list': torch.tensor(sample_t), 'sample_t': torch.tensor(sample_t)}, 'rossler_data.pth')


torch.Size([100, 11, 3])
